In [17]:
import numpy as np
from matplotlib import pyplot as plt
import sys, os, cv2, time
from src.representation.colorwhele import visualize_flow
from src.evaluation.end_point_error import calc_MEPE_directory
from src.evaluation.angular_error import calc_AAE_directory

In [18]:
def calcOpticalFlowLK(prevImg, nextImg, winSize = 39, threshD = 1e-9):
    prevImg = prevImg/1.
    nextImg = nextImg/1.
    
    #Calculate derivatives alond x and y
    prevDx = cv2.Sobel(prevImg,cv2.CV_64F,1,0, ksize=3)
    prevDy = cv2.Sobel(prevImg,cv2.CV_64F,0,1, ksize=3)
    nextDx = cv2.Sobel(nextImg,cv2.CV_64F,1,0, ksize=3)
    nextDy = cv2.Sobel(nextImg,cv2.CV_64F,0,1, ksize=3)
    
    #compromise solution by the least squares principle
    
    sigma1 = prevDx**2 + nextDx**2
    sigma2 = sigma4 = prevDx*prevDy+nextDx*nextDy
    sigma3 = (nextImg-prevImg)*(prevDx+nextDx)
    sigma5 = prevDy**2 + nextDy**2
    sigma6 = (nextImg-prevImg)*(prevDy+nextDy)
    sigma = [sigma1, sigma2, sigma3, sigma4, sigma5, sigma6]
    
    for i in range(len(sigma)):
        #apply average blurring
        sigma[i] = cv2.blur(sigma[i], (winSize, winSize))
     
    #Use Cramer's rule to solve the equation
    
    #Calculate determinant of the coefficients
    D = sigma[1]**2 - sigma[0]*sigma[4]
    D[np.abs(D) < threshD] = np.Inf

    
    #Cramers rule
    # https://pressbooks.bccampus.ca/algebraintermediate/chapter/solve-systems-of-equations-using-determinants/
    # Replace the X coefficients with the constants and take the determinant 
    hx = (sigma[4]*sigma[2]-sigma[1]*sigma[5]) / D
    # Replace the Y coefficients with the constants and take the determinant 
    hy = (sigma[1]*sigma[2]-sigma[0]*sigma[5]) / D
    
    #length -> euclidean distance
    arrow = np.sqrt(hx**2+hy**2)
    
    return hx, -hy, arrow

In [19]:
def getCoor(hx, hy, arrow, step = 20, percent = 0.1):
    y, x= np.meshgrid(np.arange(0, hx.shape[0], step), np.arange(0, hx.shape[1], step))
    coor = np.vstack((x.flatten(), y.flatten()))
    
    arrow = arrow[y, x]
    index = np.argsort(arrow.flatten())
    coor = coor[:, index[int(-len(index)*percent):]]
    return coor

In [20]:
def drawArrow(img, hx, hy, coor, scale=50):
    color = (0, 255, 255)
    mask = np.zeros_like(img)
    m = np.max((np.abs(hx), np.abs(hy)))
    if m < 0.5:
        scale /=0.5
    for i in range(coor.shape[1]):
        x1 = int(coor[0, i])
        y1 = int(coor[1, i])
        x2 = int(coor[0, i]+hx[y1, x1]*scale)
        y2 = int(coor[1, i]+hy[y1, x1]*scale)
        mask = cv2.line(mask, (x1, y1), (x2, y2), color, thickness=1)
        mask = cv2.circle(mask,(x1,y1), 3, color, -1)
    out = cv2.add(img, mask)
    return out

In [21]:
def test():
    prevName = "dataset/eval-data-gray/Army/frame07.png"; nextName = "dataset/eval-data-gray/Army/frame08.png"
    prevImg = cv2.imread(prevName)
    nextImg = cv2.imread(nextName)
    prevGray = cv2.cvtColor(prevImg, cv2.COLOR_BGR2GRAY)
    nextGray = cv2.cvtColor(nextImg, cv2.COLOR_BGR2GRAY)
    hx, hy, arrow= calcOpticalFlowLK(prevGray, nextGray)
    print("arrow_max = {}".format(np.max(arrow)))
    coor = getCoor(hx, hy, arrow, step=10, percent=0.2)
    out = drawArrow(prevImg, hx, hy, coor)
    plt.hist(arrow.ravel(), 1000, [0, 1]); plt.show()
    while 1:
        cv2.imshow('frame',out)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

In [22]:
def run(prevImg, nextImg):
    prevGray = cv2.cvtColor(prevImg, cv2.COLOR_BGR2GRAY)
    nextGray = cv2.cvtColor(nextImg, cv2.COLOR_BGR2GRAY)
    hx, hy, arrow= calcOpticalFlowLK(prevGray, nextGray)
    coor = getCoor(hx, hy, arrow)
    out = drawArrow(prevImg, hx, hy, coor)
    return out

In [23]:
def main(folder_name):
    folder_in = 'dataset/other-data-gray/'+folder_name+'/'
    folder_out = 'results/result-other-grey-LK/'+folder_name+'/'
    if not os.path.exists(folder_out):
        os.mkdir(folder_out)
    files = os.listdir(folder_in)
    files.sort()
    for i in range(len(files)-1):
        prevImg = cv2.imread(folder_in+files[i])
        nextImg = cv2.imread(folder_in+files[i+1])
        out = run(prevImg, nextImg)
        cv2.imwrite(folder_out+str(i)+'.png', out)

In [24]:
def calc_all():
    folders = ["Army","Backyard","Basketball","Dumptruck","Evergreen","Grove","Mequon","Schefflera","Teddy","Urban","Wooden","Yosemite"]
    for folder in folders:
        main(folder)
        print(folder +": Succesfully calcualted optical flow")

In [25]:
#calc_all()

In [26]:
def calc_all_other():
    folders = ["Beanbags","Dimetrodon","DogDance","Grove2","Grove3","Hydrangea","MiniCooper","RubberWhale","Urban2","Urban3","Venus","Walking"]
    for folder in folders:
        main(folder)
        print(folder +": Succesfully calcualted optical flow")

In [27]:
#calc_all_other()

In [28]:
def video(infile, outfile = 'result.avi'):
    assert os.path.exists(infile), "video doesn't exist"
    cap = cv2.VideoCapture(infile)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    out_video = cv2.VideoWriter(outfile, fourcc, fps, (int(width), int(height)))
    _, old_frame = cap.read()
    while 1:
        ret, frame = cap.read()
        if ret:
            out = run(old_frame, frame)
            cv2.imshow('frame', out)
            out_video.write(out)
        else: break
    cv2.destroyAllWindows()
    out_video.release()
    cap.release()

In [15]:
#video("/Users/xavierverbrugge/Documents/School/Master/2e Master/1 SEM/Computer Vision/Assignment 2/Forest_15_3b_Videvo.mov")

In [16]:
# Calculation of Average end-point error
# End-to-end point error is calculated by comparing an estimated optical flow vector (v_est ) with a groundtruth optical flow vector ( v_gt ).

In [37]:
def calc():
    folders = ["Dimetrodon","Grove2","Grove3","Hydrangea","RubberWhale","Urban2","Urban3","Venus"]
    #folders = ["RubberWhale","Urban2","Urban3","Venus"]

    for folder in folders:
        prevImg= cv2.imread("dataset/other-data-gray/"+folder+"/frame10.png")
        nextImg= cv2.imread("dataset/other-data-gray/"+folder+"/frame11.png")
        prevGray = cv2.cvtColor(prevImg, cv2.COLOR_BGR2GRAY)
        nextGray = cv2.cvtColor(nextImg, cv2.COLOR_BGR2GRAY)
        hx, hy, arrow= calcOpticalFlowLK(prevGray, nextGray)
        print(folder +": Succesfully calcualted optical flow")
        out = visualize_flow(u = hx, v = hy)
        #Save the flow image
        folder_out = "results/result-other-grey-LK/"+str(folder)+"/"
        if not os.path.exists(folder_out):
            os.mkdir(folder_out)
        cv2.imwrite(folder_out+ str(folder)+".png", out)

        #Replace Nan's
        u = np.nan_to_num(hx)
        v = np.nan_to_num(hy)

        
        #Calculate statistics
        mepe, sdEpe = calc_MEPE_directory(folder, u = u, v = v)
        print("The Average End Point Error for "+folder+ " is: " + str(mepe)+" and the standard deviation is: "+str(sdEpe))

        mang, sdAngular = calc_AAE_directory(folder, u= u, v = v)

        print("The Average Angular error for "+folder+ " is: " + str(mang)+" and the standard deviation is: "+str(sdAngular))

In [39]:
calc()

Dimetrodon: Succesfully calcualted optical flow
The Average End Point Error for Dimetrodon is: 1.95 and the standard deviation is: 0.71
The Average Angular error for Dimetrodon is: 0.55 and the standard deviation is: 0.13
Grove2: Succesfully calcualted optical flow
The Average End Point Error for Grove2 is: 3.03 and the standard deviation is: 0.5
The Average Angular error for Grove2 is: 0.37 and the standard deviation is: 0.06
Grove3: Succesfully calcualted optical flow
The Average End Point Error for Grove3 is: 3.84 and the standard deviation is: 2.31
The Average Angular error for Grove3 is: 0.39 and the standard deviation is: 0.2
Hydrangea: Succesfully calcualted optical flow
The Average End Point Error for Hydrangea is: 3.64 and the standard deviation is: 1.16
The Average Angular error for Hydrangea is: 0.37 and the standard deviation is: 0.13
RubberWhale: Succesfully calcualted optical flow
The Average End Point Error for RubberWhale is: 1.14 and the standard deviation is: 0.49
The